# Lake chemistry data: Preprocessing

Author: Jakob Nyström, 5563

In [99]:
import pandas as pd
import numpy as np
import pyproj

In [100]:
import jupyter_black

jupyter_black.load()

## 1. Load and inspect data

Summary: Data set contains 8974 observations and 52 columns. There are 108 lakes included, and there are between 171 and 34 measurements per lake.

In [101]:
# Load raw data
df_lake_chem = pd.read_csv("../data/LakeChem 2001-2022 Surface Season cleaned.csv")
df_lake_chem.head()

,MD-MVM Id,Nationellt övervakningsstations-ID,Övervakningsstation,Stationskoordinat N/X,Stationskoordinat E/Y,Län,Kommun,MS_CD C3,ProvId,Provdatum,...,Tot-P (µg/l P),Si (mg/l),Fe (µg/l),Al (µg/l),Al_s (µg/l),Syrgashalt (mg/l O2),Siktdjup (m),Siktdjup med kikare (m),Siktdjup utan kikare (m),Vattentemperatur (°C)
0,54,262403.0,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,22480,2001-03-28,...,8.0,2.27,40.0,NaN,85.0,NaN,5.5,NaN,NaN,0.6
1,54,262403.0,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,22481,2001-05-21,...,5.0,1.62,43.0,NaN,80.0,NaN,7.6,NaN,NaN,10.2
2,54,262403.0,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,22482,2001-08-22,...,5.0,1.73,19.0,NaN,45.0,NaN,6.0,NaN,NaN,18.6
3,54,262403.0,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,22483,2001-10-15,...,8.0,1.72,41.0,NaN,50.0,NaN,7.0,NaN,NaN,10.3
4,54,262403.0,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,22484,2002-02-26,...,8.0,1.89,36.0,NaN,55.0,NaN,NaN,NaN,NaN,1.5


In [102]:
df_lake_chem.shape

(8974, 52)

In [103]:
print(df_lake_chem["Övervakningsstation"].value_counts())

Övervakningsstation
Stora Skärsjön    171
Skärgölen         149
Hjärtsjön          87
Dagarn             87
Övre Skärsjön      87
                 ... 
Tronntjärnarna     60
Norra Reivo        60
Älgsjön            43
Fräcksjön          42
Stora Gryten       34
Name: count, Length: 108, dtype: int64


In [104]:
df_lake_chem.isnull().sum()

MD-MVM Id                                0
Nationellt övervakningsstations-ID     161
Övervakningsstation                      0
Stationskoordinat N/X                    0
Stationskoordinat E/Y                    0
Län                                      0
Kommun                                   0
MS_CD C3                               227
ProvId                                   0
Provdatum                                0
Provtagningsår                           0
Provtagningsmånad                        0
Provtagningsdag                          0
SeasonType                               0
Season                                   0
Season priority                          0
Season cleaning                          0
Max provdjup (m)                         1
TOC (mg/l C)                             0
DOC (mg/l C)                          8553
Tot-N_ps (µg/l N)                     6607
Tot-N_TNb (µg/l N)                    2375
Abs_F 254 (/5cm)                      4921
Abs_F 365 (

In [105]:
list_of_lakes = df_lake_chem["Övervakningsstation"].unique().tolist()
print(list_of_lakes)

['Spjutsjön', 'Edasjön', 'Mäsen', 'Hällsjön', 'Gipsjön', 'Översjön', 'Remmarsjön', 'Hällvattnet', 'Siggeforasjön', 'Bäen', 'Valkeajärvi', 'Valasjön', 'Skärsjön', 'Latnjajaure', 'Sännen', 'Svinarydsjön', 'Örsjön', 'Harasjön', 'Svartesjön', 'Stora Skärsjön', 'Fyrsjön', 'Älgarydssjön', 'Björken', 'Älgsjön', 'Rammsjön', 'Lilla Öresjön', 'Överudssjön', 'N. Yngern', 'Stora Envättern', 'Hjärtsjön', 'Fiolen', 'Storasjö', 'Ulvsjön', 'Bysjön', 'Dagarn', 'Övre Skärsjön', 'Hinnasjön', 'Limmingsjön', 'Granvattnet', 'Rotehogstjärnen', 'Hökesjön', 'Allgjuttern', 'Bergträsket', 'Brunnsjön', 'Tomeshultagölen', 'Stora Tresticklan', 'Brännträsket', 'St. Lummersjön', 'Vuolgamjaure', 'Stor-Arasjön', 'Öjsjön', 'Njalakjaure', 'Täftesträsket', 'Stora Gryten', 'Skärgölen', 'Grissjön', 'Alsjön', 'Degervattnet', 'Humsjön', 'Stor-Björsjön', 'Sangen', 'Stor-Backsjön', 'Västra Solsjön', 'Fräcksjön', 'Tärnan', 'Tängersjö', 'Fjärasjö', 'Louvvajaure', 'Jutsajaure', 'Övre Fjätsjön', 'Pahajärvi', 'Tväringen', 'Sidensjön

In [106]:
len(list_of_lakes)

108

## 2. Data cleaning 

We rename columns to English names and drop columns that are not needed for any analysis or joins. Coordinates are converted to standard double decimal (DD) format. Data are cast to the right types. 

In [107]:
print(list(df_lake_chem.columns))

['MD-MVM Id', 'Nationellt övervakningsstations-ID', 'Övervakningsstation', 'Stationskoordinat N/X', 'Stationskoordinat E/Y', 'Län', 'Kommun', 'MS_CD C3', 'ProvId', 'Provdatum', 'Provtagningsår', 'Provtagningsmånad', 'Provtagningsdag', 'SeasonType', 'Season', 'Season priority', 'Season cleaning', 'Max provdjup (m)', 'TOC (mg/l C)', 'DOC (mg/l C)', 'Tot-N_ps (µg/l N)', 'Tot-N_TNb (µg/l N)', 'Abs_F 254 (/5cm)', 'Abs_F 365 (/5cm)', 'Abs_F 420 (/5cm)', 'Abs_F 436 (/m)', 'Abs_OF 420 (/5cm)', 'Turb_FNU (FNU)', 'Kfyll (µg/l)', 'pH', 'Kond_25 (mS/m)', 'Alk/Acid (mekv/l)', 'Ca (mekv/l)', 'Mg (mekv/l)', 'Na (mekv/l)', 'K (mekv/l)', 'SO4 (mekv/l)', 'Cl (mekv/l)', 'F (mekv/l)', 'NH4-N (µg/l N)', 'NO2+NO3-N (µg/l N)', 'PO4-P (µg/l P)', 'Tot-P (µg/l P)', 'Si (mg/l)', 'Fe (µg/l)', 'Al (µg/l)', 'Al_s (µg/l)', 'Syrgashalt (mg/l O2)', 'Siktdjup (m)', 'Siktdjup med kikare (m)', 'Siktdjup utan kikare (m)', 'Vattentemperatur (°C)']


### Drop unwanted columns

In [108]:
# Columns that will not be used for any analysis
cols_to_drop = [
    "Nationellt övervakningsstations-ID",
    "ProvId",
    "Season priority",
    "Season cleaning",
    "Abs_F 365 (/5cm)",
    "Abs_F 436 (/m)",
    "Abs_OF 420 (/5cm)",
]

# Drop these columns from the dataframe
df_lake_chem = df_lake_chem.drop(cols_to_drop, axis="columns")
df_lake_chem.head()

,MD-MVM Id,Övervakningsstation,Stationskoordinat N/X,Stationskoordinat E/Y,Län,Kommun,MS_CD C3,Provdatum,Provtagningsår,Provtagningsmånad,...,Tot-P (µg/l P),Si (mg/l),Fe (µg/l),Al (µg/l),Al_s (µg/l),Syrgashalt (mg/l O2),Siktdjup (m),Siktdjup med kikare (m),Siktdjup utan kikare (m),Vattentemperatur (°C)
0,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-03-28,2001,3,...,8.0,2.27,40.0,NaN,85.0,NaN,5.5,NaN,NaN,0.6
1,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-05-21,2001,5,...,5.0,1.62,43.0,NaN,80.0,NaN,7.6,NaN,NaN,10.2
2,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-08-22,2001,8,...,5.0,1.73,19.0,NaN,45.0,NaN,6.0,NaN,NaN,18.6
3,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-10-15,2001,10,...,8.0,1.72,41.0,NaN,50.0,NaN,7.0,NaN,NaN,10.3
4,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2002-02-26,2002,2,...,8.0,1.89,36.0,NaN,55.0,NaN,NaN,NaN,NaN,1.5


### Rename columns

In [109]:
# Columns to be renamed
column_mapper = {
    "Övervakningsstation": "Survey station",
    "Stationskoordinat N/X": "Latitude",
    "Stationskoordinat E/Y": "Longitude",
    "Län": "County",
    "Kommun": "Municipality",
    "Provdatum": "Date",
    "Provtagningsår": "Year",
    "Provtagningsmånad": "Month",
    "Provtagningsdag": "Day",
    "Max provdjup (m)": "Max sample depth (m)",
    "Kfyll (µg/l)": "C_phyll (µg/l)",
    "Kond_25 (mS/m)": "Cond_25 (mS/m)",
    "Syrgashalt (mg/l O2)": "Oxygen (mg/l O2)",
    "Siktdjup (m)": "Secchi depth (m)",
    "Siktdjup med kikare (m)": "Secchi depth binoculars (m)",
    "Siktdjup utan kikare (m)": "Secchi depth no binoculars (m)",
    "Vattentemperatur (°C)": "Water temp (°C)",
}

df_lake_chem = df_lake_chem.rename(columns=column_mapper)
df_lake_chem.head()

,MD-MVM Id,Survey station,Latitude,Longitude,County,Municipality,MS_CD C3,Date,Year,Month,...,Tot-P (µg/l P),Si (mg/l),Fe (µg/l),Al (µg/l),Al_s (µg/l),Oxygen (mg/l O2),Secchi depth (m),Secchi depth binoculars (m),Secchi depth no binoculars (m),Water temp (°C)
0,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-03-28,2001,3,...,8.0,2.27,40.0,NaN,85.0,NaN,5.5,NaN,NaN,0.6
1,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-05-21,2001,5,...,5.0,1.62,43.0,NaN,80.0,NaN,7.6,NaN,NaN,10.2
2,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-08-22,2001,8,...,5.0,1.73,19.0,NaN,45.0,NaN,6.0,NaN,NaN,18.6
3,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2001-10-15,2001,10,...,8.0,1.72,41.0,NaN,50.0,NaN,7.0,NaN,NaN,10.3
4,54,Spjutsjön,6722638,524356,Dalarnas län,Falun,WA42559716,2002-02-26,2002,2,...,8.0,1.89,36.0,NaN,55.0,NaN,NaN,NaN,NaN,1.5


In [110]:
print(list(df_lake_chem.columns))

['MD-MVM Id', 'Survey station', 'Latitude', 'Longitude', 'County', 'Municipality', 'MS_CD C3', 'Date', 'Year', 'Month', 'Day', 'SeasonType', 'Season', 'Max sample depth (m)', 'TOC (mg/l C)', 'DOC (mg/l C)', 'Tot-N_ps (µg/l N)', 'Tot-N_TNb (µg/l N)', 'Abs_F 254 (/5cm)', 'Abs_F 420 (/5cm)', 'Turb_FNU (FNU)', 'C_phyll (µg/l)', 'pH', 'Cond_25 (mS/m)', 'Alk/Acid (mekv/l)', 'Ca (mekv/l)', 'Mg (mekv/l)', 'Na (mekv/l)', 'K (mekv/l)', 'SO4 (mekv/l)', 'Cl (mekv/l)', 'F (mekv/l)', 'NH4-N (µg/l N)', 'NO2+NO3-N (µg/l N)', 'PO4-P (µg/l P)', 'Tot-P (µg/l P)', 'Si (mg/l)', 'Fe (µg/l)', 'Al (µg/l)', 'Al_s (µg/l)', 'Oxygen (mg/l O2)', 'Secchi depth (m)', 'Secchi depth binoculars (m)', 'Secchi depth no binoculars (m)', 'Water temp (°C)']


### Convert coordinates

In [111]:
# Define a function to convert coordinates from SWEREF 99 to standard
# double decimal format
def convert_coordinates_to_dd(df):
    df_copy = df.copy()
    transformer = pyproj.Transformer.from_crs(crs_from="EPSG:3006", crs_to="EPSG:4326")

    # Note that the inputs for transform is long, lat
    df_copy["Latitude"], df_copy["Longitude"] = transformer.transform(
        df_copy["Latitude"], df_copy["Longitude"]
    )
    return df_copy

In [112]:
# Run coordinate transformation
df_lake_chem = convert_coordinates_to_dd(df_lake_chem)
df_lake_chem.head()

,MD-MVM Id,Survey station,Latitude,Longitude,County,Municipality,MS_CD C3,Date,Year,Month,...,Tot-P (µg/l P),Si (mg/l),Fe (µg/l),Al (µg/l),Al_s (µg/l),Oxygen (mg/l O2),Secchi depth (m),Secchi depth binoculars (m),Secchi depth no binoculars (m),Water temp (°C)
0,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-03-28,2001,3,...,8.0,2.27,40.0,NaN,85.0,NaN,5.5,NaN,NaN,0.6
1,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-05-21,2001,5,...,5.0,1.62,43.0,NaN,80.0,NaN,7.6,NaN,NaN,10.2
2,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-08-22,2001,8,...,5.0,1.73,19.0,NaN,45.0,NaN,6.0,NaN,NaN,18.6
3,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-10-15,2001,10,...,8.0,1.72,41.0,NaN,50.0,NaN,7.0,NaN,NaN,10.3
4,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2002-02-26,2002,2,...,8.0,1.89,36.0,NaN,55.0,NaN,NaN,NaN,NaN,1.5


### Combine N columns

There are two Tot-N columns based on two different methods, depending on when the sample was taken / analyzed. We combine these into one column.

In [113]:
# Combine old and new way of calculating Tot-N
df_lake_chem["Tot-N (µg/l N)"] = np.where(
    df_lake_chem["Tot-N_TNb (µg/l N)"].notnull(),
    df_lake_chem["Tot-N_TNb (µg/l N)"],
    df_lake_chem["Tot-N_ps (µg/l N)"],
)

# Drop the old columns
df_lake_chem = df_lake_chem.drop(
    ["Tot-N_TNb (µg/l N)", "Tot-N_ps (µg/l N)"], axis="columns"
)
df_lake_chem.head()

,MD-MVM Id,Survey station,Latitude,Longitude,County,Municipality,MS_CD C3,Date,Year,Month,...,Si (mg/l),Fe (µg/l),Al (µg/l),Al_s (µg/l),Oxygen (mg/l O2),Secchi depth (m),Secchi depth binoculars (m),Secchi depth no binoculars (m),Water temp (°C),Tot-N (µg/l N)
0,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-03-28,2001,3,...,2.27,40.0,NaN,85.0,NaN,5.5,NaN,NaN,0.6,409.0
1,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-05-21,2001,5,...,1.62,43.0,NaN,80.0,NaN,7.6,NaN,NaN,10.2,360.0
2,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-08-22,2001,8,...,1.73,19.0,NaN,45.0,NaN,6.0,NaN,NaN,18.6,195.0
3,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-10-15,2001,10,...,1.72,41.0,NaN,50.0,NaN,7.0,NaN,NaN,10.3,383.0
4,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2002-02-26,2002,2,...,1.89,36.0,NaN,55.0,NaN,NaN,NaN,NaN,1.5,385.0


### Adjust absorbance measures

In [114]:
# Adjust Abs 420 to be per m
df_lake_chem["Abs_F 420 (/m)"] = df_lake_chem["Abs_F 420 (/5cm)"] * 20
df_lake_chem = df_lake_chem.drop("Abs_F 420 (/5cm)", axis="columns")

In [115]:
# Function to do the SUVA 254 adjustments
def convert_to_suva254(df):
    df = df.copy()

    # Extract based absorbance
    abs_254 = df["Abs_F 254 (/5cm)"]

    # Convert absorbance to per 1 cm
    abs_254 = abs_254 / 5

    # Correct for Fe absorbance (incl. convert microgram to milligram)
    abs_254 = abs_254 - 0.0653 * (df["Fe (µg/l)"] / 1000) + 0.002

    # Divide by TOC concentration and convert to m
    abs_254 = (abs_254 / df["TOC (mg/l C)"]) * 100

    df["SUVA_254 (/m)"] = abs_254

    return df

In [116]:
df_lake_chem = convert_to_suva254(df_lake_chem)
df_lake_chem.head()

,MD-MVM Id,Survey station,Latitude,Longitude,County,Municipality,MS_CD C3,Date,Year,Month,...,Al (µg/l),Al_s (µg/l),Oxygen (mg/l O2),Secchi depth (m),Secchi depth binoculars (m),Secchi depth no binoculars (m),Water temp (°C),Tot-N (µg/l N),Abs_F 420 (/m),SUVA_254 (/m)
0,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-03-28,2001,3,...,NaN,85.0,NaN,5.5,NaN,NaN,0.6,409.0,1.06,NaN
1,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-05-21,2001,5,...,NaN,80.0,NaN,7.6,NaN,NaN,10.2,360.0,1.02,NaN
2,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-08-22,2001,8,...,NaN,45.0,NaN,6.0,NaN,NaN,18.6,195.0,0.58,NaN
3,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2001-10-15,2001,10,...,NaN,50.0,NaN,7.0,NaN,NaN,10.3,383.0,0.70,NaN
4,54,Spjutsjön,60.638793,15.445276,Dalarnas län,Falun,WA42559716,2002-02-26,2002,2,...,NaN,55.0,NaN,NaN,NaN,NaN,1.5,385.0,0.70,NaN


In [117]:
df_lake_chem["SUVA_254 (/m)"].mean()

2.9066259203061526

### Calculate ratios for dependent variables

TOC (total organic carbon) : TON (total organic nitrogen)
<br>
TOC : TOP (total organic phosphorus)

In [118]:
def calculate_toc_ton_ratio(df):
    df = df.copy()
    toc = df["TOC (mg/l C)"]

    # Convert TOC to mol/l
    toc = (toc / 1000) / 12.011

    # Calculate TON
    ton = df["Tot-N (µg/l N)"] - df["NH4-N (µg/l N)"] - df["NO2+NO3-N (µg/l N)"]

    # Convert TON to mol/l
    ton = (ton / 1000000) / 28.02

    # Calculate the ratio and
    df["TOC:TON (mol/l)"] = toc / ton

    return df

In [119]:
def calculate_toc_top_ratio(df):
    df = df.copy()
    toc = df["TOC (mg/l C)"]

    # Convert TOC to mol/l
    toc = (toc / 1000) / 12.011

    # Calculate TOP
    ton = df["Tot-P (µg/l P)"] - df["PO4-P (µg/l P)"]

    # Convert TOP to mol/l
    ton = (ton / 1000000) / 123.88

    # Calculate the ratio and
    df["TOC:TOP (mol/l)"] = toc / ton

    return df

In [120]:
# Calculate the ratios and return updated dataframes
df_lake_chem = calculate_toc_ton_ratio(df_lake_chem)
df_lake_chem = calculate_toc_top_ratio(df_lake_chem)

## 3. Save processed data

In [121]:
# Save the file in the data folder
df_lake_chem.to_csv("../data/lake_chem_data_clean.csv")